In [1]:
from pdfminer_testing import convert_pdf_to_text, convert_pdf_to_text_clean, split_pages
import os
import chromadb

In [2]:
# Iterate through all the PDFs in the directory and convert them to text.
# Return a list of the text from each PDF. Texts are returned in text list, 
# and the source of each text is returned in the sources list.
def get_text_from_pdfs(directory):
    text = []
    sources = [] # list of strings, each string is the name of a PDF
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            text_file= convert_pdf_to_text_clean(directory + filename)[0]
            text_file = split_pages(text_file, page_length=500, overlap=20)
            source_list = [f"{filename}_page: {i+1}" for i in range(len(text_file))]
            #print (f"source_list: {source_list}")
            text+=text_file
            sources+=source_list
            print(f"pages in {filename}: {len(text_file)}")
    return text, sources

In [3]:
text, sources = get_text_from_pdfs("sample_files/") 
print(f"Number of texts: {len(text)}")
print(f"Number of sources: {len(sources)}")

File name: sample_files/1502.03167.pdf
pages in 1502.03167.pdf: 21
File name: sample_files/2205.14135.pdf
pages in 2205.14135.pdf: 50
File name: sample_files/2301.11732.pdf
pages in 2301.11732.pdf: 23
File name: sample_files/2305.09880.pdf
pages in 2305.09880.pdf: 82
File name: sample_files/2307.12574.pdf
pages in 2307.12574.pdf: 16
Number of texts: 192
Number of sources: 192


In [ ]:
#print(text[3])

In [6]:
chroma_client = chromadb.PersistentClient(path='./chromadb')
collection = chroma_client.create_collection("test_collection")

In [5]:
chroma_client.delete_collection("test_collection")

In [7]:
# create a list of sources dictionaries
sources_list = []
for source in sources:
    sources_list.append({"source": source})

# create a list of ids for the inserted documents
ids_list = []
for i in range(len(text)):
    ids_list.append(str(i+1))
print(f"Number of ids: {len(ids_list)}")

Number of ids: 192


In [ ]:
#print(sources_list)

In [ ]:
#print(ids_list)

In [8]:
from sentence_transformers import SentenceTransformer


C:\Users\rochakchadha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

C:\Users\rochakchadha\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
embeddings = model.encode(text)

In [11]:
print(embeddings.shape)

(192, 768)


In [12]:
# insert the documents into the collection
collection.add(
    embeddings=embeddings,
    documents=text,
    ids=ids_list,
    metadatas=sources_list
)
#pretty_print(collection)

In [162]:
query = "Can you explain the roles of heterogeneous feature distillation (HFD) and bidirectional selective distillation (BSD) in improving the performance of CNN and ViT models in the context of semantic segmentation?"

In [188]:
query_list = [query]
query_embeddings = model.encode(query_list)
n_results = 4
query_results  = collection.query(
    query_embeddings = query_embeddings,
    n_results=n_results,
    include= [ "documents"]
)

In [189]:
context = ''.join(query_results['documents'][0][i] for i in range(n_results))
print(len(context.split()))

708


In [ ]:
print(context)

In [190]:
import os
import openai
openai.api_key = "EMPTY"
openai.api_base = "http://172.29.40.143:8000/v1"

In [191]:
system_message = {
    "role": "system",
    "content": "You are a helpful and honest assistant. Answer the question only based on the provided context and nothing else. If you cannot answer the question, please say I dont know."
    #"content": "You are a helpful, respectful and honest assistant. Answer the question in a helpful manner." 
}
content = f"{query} \nContext: {context}"


In [192]:
#truncate the content to 3996 words to keep the total token count below 4096
if len(content.split()) > 3996:
    content = ' '.join(content.split()[:3996])

print(len(content.split()))

732


In [ ]:
user_message = {"role": "user", "content": content}

messages = [system_message, user_message]
response = openai.ChatCompletion.create(
    model="Llama-2-7b-chat-hf",
    temperature=0.9,
    max_tokens=4000,
    messages=messages,
    stream=True,
)

for chunk in response:
    content = chunk["choices"][0]["delta"].get("content", "")
    print(content, end="", flush=True)
print()

#TODO:
1. Move chromadb as a server to ASE Pro2 node
2. implement as a function to load files from the ASE pro 2 server and create embeddings